In [ ]:
# Instala las librerías contenidas en el archivo requirements. La bandera -q es para que no muestre los mensajes de instalación.
%pip install -r ./requirements.txt -q

💡 Es útil emplear la herramienta de `nbdev`, debido a que permiten utilizar git sin enviar los metadatos de las celdas de un notebook. Esto es útil para evitar conflictos en los notebooks cuando se trabaja en equipo.

In [96]:
!nbdev_install_hooks

Hooks are installed.


In [ ]:
# Load pandas

import pandas as pd
import scrapy
from scrapy.selector import SelectorList
import os

In [ ]:
# Leer archivo HTML, que contiene los nombrs de las variables de nuestro dataset
# y seleccionar las filas de la tabla

ruta = './variables.html'
doc = scrapy.Selector(
    text=open(ruta, 'r', encoding='utf-8').read()
)
# Para ver la razón de por qué usar esta expresión de xpath, ver: https://devhints.io/xpath
filas_tabla = doc.xpath("//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]")
print(filas_tabla)

In [91]:
"""
La estructura de una fila de la tabla es más o menos así:

<div>
    <div>
        <form></form>
        <span></span>
        <a>
            Nombre_de_la_variable_en_dataset
        </a>
    </div>
</div>
<div>
    <div>
        <p>
            <em>
                Nombre_completo_variable
            </em>
        </p>
        <p>
            Descripción_variable
        </p>
    </div>
    <div>
    </div>
</div>
"""

def formatear_texto(texto: str):

    """
    Reemplaza cadenas de espacios en blanco por un solo espacio
    y saltos de línea por un espacio.
    """

    return ' '.join(texto.split()).replace('\n', ' ').strip()

def parsear_variables(variables: SelectorList):
    
    for fila in filas_tabla:

        nombre_var_dataset = fila.css('a::text').get().strip()
        nombre_var_completo = fila.css('em::text').get().strip()
        nombre_var_completo = f'\"{formatear_texto(nombre_var_completo)}\"'
        descripcion_var = fila.css('p')[1].css('::text').get().strip()
        descripcion_var = f'\"{formatear_texto(descripcion_var)}\"'

        yield (nombre_var_dataset, nombre_var_completo, descripcion_var)
        

In [ ]:
ruta_archivo_variables = './datasets/variables.csv'

# Crea un archivo con las variables a partir del generador
def crear_archivo_variables(variables: SelectorList):
    
    with open(ruta_archivo_variables, 'w') as archivo:

        # Escribe el encabezado
        archivo.write('id,nombre,desc\n')

        # Escribe las variables
        for variable in parsear_variables(variables):
            archivo.write(','.join(variable) + '\n')

In [92]:
# Crea el archivo si no existe
if not os.path.exists(ruta_archivo_variables):
    
    os.makedirs(
        os.path.dirname(
            ruta_archivo_variables
        ),
        exist_ok=True
    )

# Escribe las variables en el archivo
crear_archivo_variables(filas_tabla)